<a href="https://colab.research.google.com/github/khs6466/080228/blob/master/%EB%94%A5%ED%8E%98%EC%9D%B4%ED%81%AC%EC%98%81%EC%83%81%EB%A7%8C%EB%93%A4%EA%B8%B0_%EC%83%8C%ED%94%84%EB%9E%80%EC%83%98%EB%B0%95%EC%82%ACAI_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 누구나 5분안에 사진한장으로 딥페이크 영상 만들기 1편: 실습(영상에 내 사진 입히기)
안녕하세요. 이 페이지는 [샌프란샘박사AI 유투브 채널](https://bit.ly/3iuwShR)에서 제공하는 **누구나 5분안에 사진 한장으로 딥페이크 영상만들기** 코드입니다. 이 페이지는 2019년 11월에 NeuroIPS에서 발표된 *First Order Motion Model for Image Animation* 페이퍼에서 발표된 기술&코드를 포함하고 있습니다. 

## 1단계: 코드 불러오기 


In [ ]:
!pip install scikit-video
!git clone https://github.com/drminix/first-order-model
%cd /content/first-order-model

     |████████████████████████████████| 2.3 MB 5.3 MB/s 
Cloning into 'first-order-model'...
remote: Enumerating objects: 286, done.
remote: Total 286 (delta 0), reused 0 (delta 0), pack-reused 286
Receiving objects: 100% (286/286), 117.26 MiB | 22.93 MiB/s, done.
Resolving deltas: 100% (133/133), done.
Checking out files: 100% (63/63), done.
/content/first-order-model


##2단계: 인공지능 신경망 파일 가져오기


영상제작에 필요한 사진, 동영상 및 인공지능 신경망 파일을 구글 드라이브로 공유 해 놓았습니다. 아래 폴더를 본인의 구글 드라이브에 연결해 주세요.

https://drive.google.com/drive/folders/1TIjufuJdPgkzvCQ3G1mICQZVCxWbCOFX?usp=sharing


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
target_folder = "/content/gdrive/My Drive/deepfake"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 3단계: 입력영상과 사진을 불러오기

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os
import skvideo.io
warnings.filterwarnings("ignore")

#입력사진
source_image = imageio.imread('/content/gdrive/MyDrive/deepfake/iu_01.png')

#입력영상
driving_video = skvideo.io.vread('/content/gdrive/My Drive/deepfake/damedame.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


def display(source, driving, generated=None):
     fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

     ims = []
     for i in range(len(driving)):
         cols = [source]
         cols.append(driving[i])
         if generated is not None:
             cols.append(generated[i])
         im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
         plt.axis('off')
         ims.append([im])

     ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
     plt.close()
     return ani
    
HTML(display(source_image, driving_video).to_html5_video())

## 5단계: 영상 만들기

In [ ]:
from demo import load_checkpoints
import os
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=os.path.join(target_folder,'vox-cpk.pth.tar'))

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 836/836 [02:37<00:00,  5.31it/s]


## 6단계: 얼굴을 추출해서 딥페이크 영상을 만들기

In [ ]:
!mkdir face_images
!mkdir input_images

In [ ]:
!python face_extractor.py

64053248/64040097 [==============================] - 1s 0us/step


In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

source_image = imageio.imread('/content/first-order-model/face_images/KakaoTalk_20220315_223723921_03.png')
source_image = resize(source_image, (256, 256))[..., :3]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 836/836 [02:37<00:00,  5.31it/s]
